In [1]:
import pygame
import math
from queue import PriorityQueue
import time
import numpy as np
import functools
from numba import jit

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BROWN = (133, 2, 1)
VISIT1 = (0, 255, 230)
VISIT2 = (0, 80, 230)
VISIT3 = (1, 106, 106)
LOOK = (0, 255, 164)
START = (217, 1, 183)
END = RED
GREY = (63, 63, 63)
OPEN = (0, 255, 149)
OPEN1 = (0, 106, 0)
PATH1 = (255, 200, 0)
PATH2 = (255, 254, 0)
PATH3 = (0, 200, 0)
BG_COLOR = (6, 69, 96)
BUTTON_COLOR = (19, 175, 240)
SCREEN_COLOR = (204, 230, 255)
VISITED = []

In [3]:
class Node:
    def __init__(self, row, col, width, total_rows):
        self.last = pygame.time.get_ticks()
        self.cooldown = 300 
        self.row = row
        self.col = col
        self.x = row*width
        self.y = col*width
        self.color = WHITE
        self.neighbors = []
        self.width = width
        self.total_rows = total_rows
        self.dec_animation = False
        self.weight = False
        
    def get_pos(self):
        return self.row, self.col
    
    def is_visited(self):
        return self.color == VISIT1
    
    def is_open(self):
        return self.color == OPEN
    
    def is_barrier(self):
        return self.color == BLACK
    
    def is_weight(self):
        return self.weight
    
    def is_start(self):
        return self.color == START
    
    def is_end(self):
        return self.color == END
    
    def is_neutral(self):
        return self.color == WHITE
    
    def is_looked(self):
        return self.color == LOOK
    
    def reset(self):
        self.color = WHITE
        self.weight = False
    
    def make_visit(self):
        if not self.is_weight():
            self.color = VISIT2
        else:
            self.color = VISIT3
        
        
    def make_open(self):
        if not self.is_weight():
            self.color = OPEN
        else:
            self.color = OPEN1
    
    def make_barrier(self):
        if not self.is_start() and not self.is_end():
            self.color = BLACK
            self.weight = False
            
    def make_weight(self):
        if not self.is_start() and not self.is_end():
            self.color = BROWN
            self.weight = True
    
    def make_end(self):
        self.color = END
        self.weight = False
    
    def make_start(self):
        self.color = START
        self.weight = False
        
    def make_path(self):
        if not self.is_weight():
            self.color = PATH1
        else:
            self.color = PATH3
    
    def looking_at(self):
        self.color = LOOK
    
    def draw(self, win):
        try:
            pygame.draw.rect(win, self.color, (self.x, self.y, self.width, self.width))
        except:
            pygame.draw.rect(win, WHITE, (self.x, self.y, self.width, self.width))

    def update_neighbors(self, grid):
        r = self.row 
        c = self.col
        if r < self.total_rows-1 and not grid[r+1][c].is_barrier():
            self.neighbors.append(grid[r+1][c])
            
        if r > 0 and not grid[r-1][c].is_barrier():
            self.neighbors.append(grid[r-1][c])
            
        if c < self.total_rows-1 and not grid[r][c+1].is_barrier():
            self.neighbors.append(grid[r][c+1])
            
        if c > 0 and not grid[r][c-1].is_barrier():
            self.neighbors.append(grid[r][c-1])
    
    def __lt__(self, other):
        return False


In [4]:
class button():
    def __init__(self, x, y,width,height, text=''):
        self.color = BUTTON_COLOR
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.text = text

    def draw(self,win,outline=None):
        #Call this method to draw the button on the screen
        if outline:
            pygame.draw.rect(win, outline, (self.x-2,self.y-2,self.width+4,self.height+4),0)
            pygame.draw.circle(win, outline, (self.x, self.y+self.height//2), self.height//2+2)
            pygame.draw.circle(win, outline, (self.x+self.width, self.y+self.height//2), self.height//2+2)
            
        pygame.draw.rect(win, self.color, (self.x,self.y,self.width,self.height),0)
        pygame.draw.circle(win, self.color, (self.x, self.y+self.height//2), self.height//2)
        pygame.draw.circle(win, self.color, (self.x+self.width, self.y+self.height//2), self.height//2)
        
        if self.text != '':
            font = pygame.font.SysFont('comicsans', 35)
            text = font.render(self.text, 1, (0,0,0))
            win.blit(text, (self.x + (self.width/2 - text.get_width()/2), self.y + (self.height/2 - text.get_height()/2)))

    def is_hover(self, pos):
        #Pos is the mouse position or a tuple of (x,y) coordinates
        if pos[0] > self.x-self.height//2 and pos[0] < self.x + self.width+self.height//2:
            if pos[1] > self.y and pos[1] < self.y + self.height:
                return True
            
        return False

In [5]:
class screen():
    def __init__(self, x,y,width,height, text=''):
        self.color = SCREEN_COLOR
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.label1 = ""
        self.text1 = text
        self.text2 = ""
        self.text3 = ""
        
    def set_label1(self, label):
        self.label1 = label
    
    def set_text1(self, text):
        self.text1 = text
    def set_text2(self, text):
        self.text2 = text
    def set_text3(self, text):
        self.text3 = text
        
    def get_text1(self):
        return self.text1

    def draw(self,win,outline=None):
        #Call this method to draw the button on the screen
        if outline:
            pygame.draw.rect(win, outline, (self.x-2,self.y-2,self.width+4,self.height+4),0)
            
        pygame.draw.rect(win, self.color, (self.x,self.y,self.width,self.height),0)
        
        if self.label1 != '':
            font = pygame.font.SysFont('comicsans', 20)
            label = font.render(self.label1, 1, (0,0,0))
            win.blit(label, (self.x + 10, self.y + 10))
        
        if self.text1 != '':
            font = pygame.font.SysFont('comicsans', 30)
            text = font.render(self.text1, 1, (0,0,0))
            win.blit(text, (self.x + (self.width/2 - text.get_width()/2), self.y + (self.height/2 - text.get_height()/2) - 50))
        if self.text2 != '':
            font = pygame.font.SysFont('comicsans', 30)
            text = font.render(self.text2, 1, (0,0,0))
            win.blit(text, (self.x + (self.width/2 - text.get_width()/2), self.y + (self.height/2 - text.get_height()/2)))
        if self.text3 != '':
            font = pygame.font.SysFont('comicsans', 30)
            text = font.render(self.text3, 1, (0,0,0))
            win.blit(text, (self.x + (self.width/2 - text.get_width()/2), 50 + self.y + (self.height/2 - text.get_height()/2)))

    def is_hover(self, pos):
        #Pos is the mouse position or a tuple of (x,y) coordinates
        if pos[0] > self.x and pos[0] < self.x + self.width:
            if pos[1] > self.y and pos[1] < self.y + self.height:
                return True
            
        return False

In [6]:
def h(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return abs(x1 - x2) + abs(y1 - y2)

In [7]:
def visit_animation2(node):
    if node.color == VISIT1:
        return True
    else:
        r, g, b = node.color
        b += 1
        node.color = (r, g, b)
        return False

In [8]:
def visit_animation(visited):
    for node in visited:
        if node.color == VISIT1 or node.color == VISIT3:
            visited.remove(node)
            continue
        r, g, b = node.color
        g += 1
        node.color = (r, g, b)
        

In [9]:
# def visit_animation(visited):
#     for node in visited:
#         if node.color[1] == 255:
#             if visit_animation2(node):
#                 visited.remove(node)
#         else:
#             r, g, b = node.color
#             g += 1
#             b-=1
#             node.color = (r, g, b)

In [10]:
def path_animation(path):
    for node in path:
        if not node.is_start():
            r, g, b = node.color
            if node.dec_animation:
                g -= 1
                if g <= PATH1[1]:
                    node.dec_animation = False
            else:
                g += 1
                if g >= PATH2[1]:
                    node.dec_animation = True
#         for _ in range(300):
#             _
        node.color = (r, g, b)

In [11]:
def reconstruct_path(came_from, start, current, draw, visited,  win, width, grid, is_draw = True):
    path = []
    c = 0
    while current in came_from:
        visit_animation(visited)
        current = came_from[current]
        if current.is_weight():
            c+= 5
        else:
            c+=1
        if current in visited:
            visited.remove(current)
        if current != start:
            path.insert(0, current)
        current.make_path()
#         path_animation(path)
        if is_draw:
            for rows in grid:
                for node in rows:
                    node.draw(win)
            draw_grid(win, len(grid), width)
            pygame.display.update()
    return path, c-1

In [12]:
def A_star(draw, grid, start, end, output, win, width):
    count = 0
    vis = 0
    open_set = PriorityQueue()
    open_set.put((0, count, start))
    came_from = {}
    g_score = {node: float("inf") for row in grid for node in row}
    g_score[start] = 0
    f_score = {node: float("inf") for row in grid for node in row}
    f_score[start] = h(start.get_pos(), end.get_pos())
    visited = []
    nebrs = []
    
    open_set_hash = {start}
    
    while not open_set.empty():
#         li = [i for i in end.neighbors if i.is_neutral()]
#         if len(li):
#             end.reset()
#             end = li[np.random.randint(len(li))]
#             end.make_end()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
        
        current = open_set.get()[2]
        open_set_hash.remove(current)
        
        if current == end:
            path, inc = reconstruct_path(came_from, start, end, draw, visited, win, width, grid)
            start.make_start()
            output.set_text1(f"Path Length: {inc}")
            output.set_text2(f"#Visited nodes: {vis}")
            if vis != 0:
                output.set_text3(f"Efficiency: {np.round(inc/vis, decimals=3)}")
            return visited, path
        c = 1      
        for neighbor in current.neighbors:
            if not neighbor.is_barrier():
                if neighbor.is_weight():
                    c = 5

                temp_g_score = g_score[current] + c
                temp_f_score = temp_g_score + h(neighbor.get_pos(), end.get_pos())
                if temp_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = temp_g_score
                    f_score[neighbor] = temp_f_score
                    if neighbor not in open_set_hash:
                        count+=1
                        open_set.put((f_score[neighbor], count, neighbor))
                        open_set_hash.add(neighbor)
                    if neighbor != end:
                        nebrs.append(neighbor)
                        neighbor.make_open()
        
        if current != start:
            vis+=c
            visited.append(current)
            current.make_visit()
        
        visit_animation(visited)
#         nebr_animation(nebrs)
        for rows in grid:
            for node in rows:
                node.draw(win)
        draw_grid(win, len(grid), width)
        pygame.display.update()
            
    return visited, False

In [13]:
def A_star_active(draw, grid, start, end, output, win, width):
    count = 0
    vis = 0
    open_set = PriorityQueue()
    open_set.put((0, count, start))
    came_from = {}
    g_score = {node: float("inf") for row in grid for node in row}
    g_score[start] = 0
    f_score = {node: float("inf") for row in grid for node in row}
    f_score[start] = h(start.get_pos(), end.get_pos())
    visited = []
    nebrs = []
    
    open_set_hash = {start}
    
    while not open_set.empty():
#         li = [i for i in end.neighbors if i.is_neutral()]
#         if len(li):
#             end.reset()
#             end = li[np.random.randint(len(li))]
#             end.make_end()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
        
        current = open_set.get()[2]
        open_set_hash.remove(current)
        
        if current == end:
            path, inc = reconstruct_path(came_from, start, end, draw, visited, win, width, grid)
            start.make_start()
            output.set_text1(f"Path Length: {inc}")
            output.set_text2(f"#Visited nodes: {vis}")
            if vis != 0:
                output.set_text3(f"Efficiency: {np.round(inc/vis, decimals=3)}")
            return visited, path
        
        c = 1
        for neighbor in current.neighbors:
            if not neighbor.is_barrier():
                if neighbor.is_weight():
                    c = 5

                temp_g_score = g_score[current] + c
                temp_f_score = temp_g_score + h(neighbor.get_pos(), end.get_pos())
                if temp_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = temp_g_score
                    f_score[neighbor] = temp_f_score
                    if neighbor not in open_set_hash:
                        count+=1
                        open_set.put((f_score[neighbor], count, neighbor))
                        open_set_hash.add(neighbor)
                    if neighbor != end:
                        nebrs.append(neighbor)
                        neighbor.make_open()
        
        if current != start:
            vis+=c
            visited.append(current)
            current.make_visit()
        
        visit_animation(visited)
#         nebr_animation(nebrs)
            
    return visited, False

In [14]:
def moded_A_star(draw, grid, start, end, output, win, width, moving_target=False):
    count = 0
    vis = 0
    open_set = PriorityQueue()
    open_set.put((0, count, start))
    came_from = {}
    g_score = {node: float("inf") for row in grid for node in row}
    g_score[start] = 0
    f_score = {node: float("inf") for row in grid for node in row}
    f_score[start] = h(start.get_pos(), end.get_pos())
    visited = []
    nebrs = []
    
    open_set_hash = {start}
    
    while not open_set.empty():
        if moving_target:
            li = [i for i in end.neighbors if i.is_neutral()]
            if len(li):
                end.reset()
                end = li[np.random.randint(len(li))]
                end.make_end()
                for rows in grid:
                    for node in rows:
                        node.draw(win)
                        draw_grid(win, len(grid), width)
                        pygame.display.update()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
        
        current = open_set.get()[2]
        open_set_hash.remove(current)
        
        if current == end:
            path, inc = reconstruct_path(came_from, start, end, draw, visited, win, width, grid)
            start.make_start()
            output.set_text1(f"Path Length: {inc}")
            output.set_text2(f"#Visited nodes: {vis}")
            if vis != 0:
                output.set_text3(f"Efficiency: {np.round(inc/vis, decimals=3)}")
            return visited, path
        
        c = 1
        for neighbor in current.neighbors:
            if not neighbor.is_barrier() and (neighbor.is_neutral() or neighbor == end or neighbor.color == BROWN):
                if neighbor.is_weight():
                    c = 5
                    
                temp_g_score = g_score[current] + c
                temp_f_score = (h(neighbor.get_pos(), end.get_pos())) * temp_g_score
                if temp_f_score < f_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = temp_g_score
                    f_score[neighbor] = temp_f_score
                    if neighbor not in open_set_hash:
                        count+=1
                        open_set.put((f_score[neighbor], count, neighbor))
                        open_set_hash.add(neighbor)
                        if neighbor != end:
                            nebrs.append(neighbor)
                            neighbor.make_open()

        if current != start:
            if current not in visited:
                visited.append(current)
                vis+=c
                current.make_visit()

        visit_animation(visited)
#         nebr_animation(nebrs)
        for rows in grid:
            for node in rows:
                node.draw(win)
        draw_grid(win, len(grid), width)
        pygame.display.update()
            
    return visited, False

In [15]:
def DFA_star(draw, grid, start, end, output, win, width, moving_target=False):
    count = 0
    vis = 0
    open_set = PriorityQueue()
    open_set.put((0, count, start))
    came_from = {}
    g_score = {node: float("inf") for row in grid for node in row}
    g_score[start] = 0
    f_score = {node: float("inf") for row in grid for node in row}
    f_score[start] = h(start.get_pos(), end.get_pos())
    visited = []
    nebrs = []
    
    open_set_hash = {start}
    
    while not open_set.empty():
        if moving_target:
            li = [i for i in end.neighbors if i.is_neutral()]
            if len(li):
                end.reset()
                end = li[np.random.randint(len(li))]
                end.make_end()
                for rows in grid:
                    for node in rows:
                        node.draw(win)
                        draw_grid(win, len(grid), width)
                        pygame.display.update()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
        
        current = open_set.get()[2]
        open_set_hash.remove(current)
        
        if current == end:
            path, inc = reconstruct_path(came_from, start, end, draw, visited, win, width, grid)
            start.make_start()
            output.set_text1(f"Path Length: {inc}")
            output.set_text2(f"#Visited nodes: {vis}")
            if vis != 0:
                output.set_text3(f"Efficiency: {np.round(inc/vis, decimals=3)}")
            return visited, path
        
        c = 1
        for neighbor in current.neighbors:
            if not neighbor.is_barrier() and (neighbor.is_neutral() or neighbor == end or neighbor.color == BROWN):
                if neighbor.is_weight():
                    c = 5
                temp_g_score = g_score[current] + c
                temp_f_score = (h(neighbor.get_pos(), end.get_pos()))-temp_g_score
                if temp_f_score < f_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = temp_g_score
                    f_score[neighbor] = temp_f_score
                    if neighbor not in open_set_hash:
                        count+=1
                        open_set.put((f_score[neighbor], count, neighbor))
                        open_set_hash.add(neighbor)
                    if neighbor != end:
                        nebrs.append(neighbor)
                        neighbor.make_open()
        
        if current != start:
            if current in visited:
                visited.remove(current)
            visited.append(current)
            vis+=c
            current.make_visit()
        
        visit_animation(visited)
#         nebr_animation(nebrs)
        for rows in grid:
            for node in rows:
                node.draw(win)
        draw_grid(win, len(grid), width)
        pygame.display.update()
            
    return visited, False

In [16]:
def IDA_star(draw, win, width, output,  grid, start, end, threshold=100, moving_target= False, visited_old = []):
    if threshold < len(grid)**2:
        count = 0
        vis = 0
        open_set = PriorityQueue()
        open_set.put((0, count, start))
        came_from = {}
        g_score = {node: float("inf") for row in grid for node in row}
        g_score[start] = 0
        f_score = {node: float("inf") for row in grid for node in row}
        f_score[start] = h(start.get_pos(), end.get_pos())

        visited = []
        nebrs = []
        while not open_set.empty():
            if moving_target:
                li = [i for i in end.neighbors if i.is_neutral()]
                if len(li):
                    end.reset()
                    end = li[np.random.randint(len(li))]
                    end.make_end()
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()

            current = open_set.get()[2]

            if current == end:
                path, inc = reconstruct_path(came_from, start, end, draw, visited, win, width, grid)
                start.make_start()
                output.set_text1(f"Path Length: {inc}")
                output.set_text2(f"#Visited nodes: {vis}")
                if vis != 0:
                    output.set_text3(f"Efficiency: {np.round(inc/vis, decimals=3)}")
                return visited, path
            
            c = 1
            if f_score[current] <= threshold:
                for neighbor in current.neighbors:
                    if not neighbor.is_barrier():
                        if neighbor.is_weight():
                            c = 5
                        temp_g_score = g_score[current] + c
                        temp_f_score = temp_g_score + h(neighbor.get_pos(), end.get_pos())
                        if temp_f_score < f_score[neighbor]:
                            came_from[neighbor] = current
                            g_score[neighbor] = temp_g_score
                            f_score[neighbor] = temp_f_score
                            count+=1
                            open_set.put((f_score[neighbor], count, neighbor))
                            if neighbor != end:
                                nebrs.append(neighbor)
                                neighbor.make_open()

            if current != start:
                visited.append(current)
                vis+=c
                current.make_visit()

            visit_animation(visited)
            for rows in grid:
                for node in rows:
                    node.draw(win)
            draw_grid(win, len(grid), width)
            pygame.display.update()
            
        if visited == visited_old:
            return visited, False
        return IDA_star(draw, win, width, output ,grid, start, end, threshold+10, moving_target, visited)
    return [], False

In [17]:
def is_free(grid, x, y):
    count = 0
    if y+1 < len(grid) and grid[x][y+1].is_barrier() :
        count +=1
    if y-1>=0 and grid[x][y-1].is_barrier():
        count +=1
    if x+1 < len(grid) and grid[x+1][y].is_barrier():
        count+=1
    if x-1>=0 and grid[x-1][y].is_barrier():
        count+=1
    if count >= 3:
        return True
    return False

In [18]:
def unvisited_n(grid, x, y):
    n = []
    if y+1 < len(grid) and grid[x][y+1].is_barrier() and is_free(grid, x, y+1):
        n.append((x, y+1))
    if y-1>=0 and grid[x][y-1].is_barrier() and is_free(grid, x, y-1):
        n.append((x, y-1))
    if x+1 < len(grid) and grid[x+1][y].is_barrier() and is_free(grid, x+1, y):
        n.append((x+1, y))
    if x-1>=0 and grid[x-1][y].is_barrier() and is_free(grid, x-1, y):
        n.append((x-1, y))
    return n

In [19]:
def make_black(grid, win):
    for row in grid:
        for node in row:
            node.make_barrier()
            node.draw(win)
    pygame.display.update()

In [20]:
def dfs_maze(draw, width, grid, start, end, left, right, top, bottom, win, vertical=True):
    make_black(grid, win)
    x, y = 1, 1
    head = grid[x][y]
    head.looking_at()
    stack = [(x, y)]
    while True:
        neighbors = unvisited_n(grid, x, y)
        if len(neighbors) > 0:
            random_index = np.random.randint(len(neighbors))
            x, y = neighbors[random_index]
            head = grid[x][y]
            head.looking_at()
            stack.append((x, y))
            head.draw(win)
            draw_grid(win, len(grid), width)
            pygame.display.update()
        else:
            if len(stack) > 0:
                x, y = stack.pop()
                grid[x][y].reset()
                grid[x][y].draw(win)
                draw_grid(win, len(grid), width)
                pygame.display.update()
            if len(stack) > 0:
                x, y = stack[-1]
#                 draw()
            else:
                break

In [21]:
def is_open_up(grid, li, win, width):
    n = []
    for x, y in li:
        if grid[x][y-1].is_neutral():
            n.append(grid[x][y])
    if len(n)>0:
        node = np.random.choice(n)
        node.reset()
        node.draw(win)
        draw_grid(win, len(grid), width)
        pygame.display.update()

In [22]:
def sidewinder_maze(draw, width, grid, start, end, left, right, top, bottom, win, vertical=True):
    make_black(grid, win)
    for x in range(1, right-1):
        grid[x][1].reset()
        grid[x][1].draw(win)
        draw_grid(win, len(grid), width)
        pygame.display.update()
        
    for y in range(3, bottom, 2):
        x = 1
        next_row = False
        while not next_row:
            lim = x+5
            if lim >= right-1:
                x_lim = right-1
                next_row = True
            else:
                x_lim = np.random.randint(x+2, lim)
            li = []
            
            for i in range(x, x_lim):
                li.append((i, y-1))
                grid[i][y].reset()
                grid[i][y].draw(win)
                draw_grid(win, len(grid), width)
                pygame.display.update()
                
            is_open_up(grid, li)
            grid[i][y].draw(win)
            draw_grid(win, len(grid), width)
            pygame.display.update()
            x = x_lim+1
            
    for x in range(1, right-1):
        grid[x][0].make_barrier()
        grid[x][0].draw(win)
        draw_grid(win, len(grid), width)
        pygame.display.update()

In [23]:
def moded_sidewinder_maze(draw, width, grid, start, end, left, right, top, bottom, win, vertical=True):
    make_black(grid, win)
    for x in range(1, right-1):
        grid[x][0].reset()
        grid[x][0].draw(win)
        draw_grid(win, len(grid), width)
        pygame.display.update()
        
    for y in range(2, bottom, 2):
        x = 1
        next_row = False
        while not next_row:
            lim = x+5
            if lim >= right-1:
                x_lim = right-1
                next_row = True
            else:
                x_lim = np.random.randint(x+2, lim)
            li = []
            
            for i in range(x, x_lim):
                li.append((i, y-1))
                grid[i][y].reset()
                grid[i][y].draw(win)
                draw_grid(win, len(grid), width)
                pygame.display.update()
            if (y == 2 and not np.random.randint(0, 5)) or (not next_row and not np.random.randint(0, 5)):
                grid[x_lim][y].reset()
                grid[x_lim][y].draw(win)
                draw_grid(win, len(grid), width)
                pygame.display.update()
                
            is_open_up(grid, li, win, width)
            x = x_lim+1
    for x in range(1, right-1):
        grid[x][0].make_barrier()
        grid[x][0].draw(win)
        draw_grid(win, len(grid), width)
        pygame.display.update()

In [24]:
def recursive_maze(draw, width, grid, start, end, left, right, top, bottom, win):
    if right-left >= 1 and bottom-top >=1:
        vertical = True
        test = False
        if right-left >= bottom-top:
            if bottom-top == 1:
                if (bottom < len(grid) and grid[left][bottom].is_barrier()) and (top-1 >= 0 and grid[left][top-1].is_barrier()):
                    vertical = False
            if vertical == True:
                l = left
                r = right
                if left-1>=0 and grid[left-1][top].is_barrier():
                    l = left+1
                if right<50 and grid[right][top].is_barrier():
                    r = right-1
                if l >= r:
                    vertical = False 
            if vertical == True:
                test = True
                rand = np.random.randint(l, r)
                if top<bottom and ((bottom - top <= 3) or (top-1 >= 0 and grid[left][top-1].is_neutral() and bottom < len(grid) and grid[left][bottom].is_neutral())):
                    br = bottom
                if top+1 < bottom-1:
                    br = np.random.randint(top+1, bottom-1)
                elif top+1 == bottom-1:
                    br = top+1
                else:
                    br = bottom
                
        elif right-left <= bottom-top:
            if right-left == 1:
                if (right < len(grid) and grid[right][top].is_barrier()) and (left-1 >= 0 and grid[left-1][top].is_barrier()):
                    return 
            t = top
            b = bottom
            if top-1>= 0 and grid[left][top-1].is_barrier():
                t = top+1
            if bottom < 50 and grid[left][bottom].is_barrier():
                b = bottom-1
            if t >= b:
                return
            test = True
            rand = np.random.randint(t, b)
            if left<right and ((right - left<= 3) or (left-1 >= 0 and grid[left-1][bottom].is_neutral() and right < len(grid) and grid[right][bottom].is_neutral())):
                    br = bottom
            if left+1 < right-1:
                br = np.random.randint(left+1, right-1)
            elif left+1 == right-1:
                br = left+1
            else:
                br = right
            vertical = False
        if test == False:
            return
        prob = 3
        if vertical:
            for y in range(top, br):
                grid[rand][y].make_barrier()
                grid[rand][y].draw(win)
                draw_grid(win, len(grid), width)
                pygame.display.update()
                
            if not np.random.randint(prob) and br - top <= 2:
                grid[rand][y+1].make_barrier()
                grid[rand][y+1].draw(win)
                draw_grid(win, len(grid), width)
                pygame.display.update()
                
            for y in range(br+1, bottom):
                grid[rand][y].make_barrier()
                grid[rand][y].draw(win)
                draw_grid(win, len(grid), width)
                pygame.display.update()
        else:
            for x in range(left, br):
                grid[x][rand].make_barrier()
                grid[x][rand].draw(win)
                draw_grid(win, len(grid), width)
                pygame.display.update()
                
            if not np.random.randint(prob) and br - left <= 2:
                grid[x+1][rand].make_barrier()
                grid[x+1][rand].draw(win)
                draw_grid(win, len(grid), width)
                pygame.display.update()
                
            for x in range(br+1, right):
                grid[x][rand].make_barrier()
                grid[x][rand].draw(win)
                draw_grid(win, len(grid), width)
                pygame.display.update()
        
        if vertical:
            recursive_maze(draw, width, grid, start, end, rand+1, right, top, br, win)
            recursive_maze(draw, width, grid, start, end, left, rand, top, br, win)
            recursive_maze(draw, width, grid, start, end, rand+1, right, br+1, bottom, win)
            recursive_maze(draw, width, grid, start, end, left, rand, br+1, bottom, win)
        else:
            recursive_maze(draw, width, grid, start, end, left, br, top, rand, win)
            recursive_maze(draw, width, grid, start, end, left, br, rand+1, bottom, win)
            recursive_maze(draw, width, grid, start, end, br+1, right, top, rand, win)
            recursive_maze(draw, width, grid, start, end, br+1, right, rand+1, bottom, win)

In [25]:
def recursive_weight_maze(draw, width, grid, start, end, left, right, top, bottom, win):
    weighted = []
    if right-left >= 1 and bottom-top >=1:
        vertical = True
        test = False
        if right-left >= bottom-top:
            if bottom-top == 1:
                if (bottom < len(grid) and grid[left][bottom].is_weight()) and (top-1 >= 0 and grid[left][top-1].is_weight()):
                    vertical = False
            if vertical == True:
                l = left
                r = right
                if left-1>=0 and grid[left-1][top].is_weight():
                    l = left+1
                if right<50 and grid[right][top].is_weight():
                    r = right-1
                if l >= r:
                    vertical = False 
            if vertical == True:
                test = True
#                 print("--->", l, r)
                rand = np.random.randint(l, r)
                if top<bottom and ((bottom - top <= 3) or (top-1 >= 0 and grid[left][top-1].is_neutral() and bottom < len(grid) and grid[left][bottom].is_neutral())):
                    br = bottom
                if top+1 < bottom-1:
                    br = np.random.randint(top+1, bottom-1)
                elif top+1 == bottom-1:
                    br = top+1
                else:
                    br = bottom
                
        elif right-left <= bottom-top:
            if right-left == 1:
                if (right < len(grid) and grid[right][top].is_weight()) and (left-1 >= 0 and grid[left-1][top].is_weight()):
                    return weighted
            t = top
            b = bottom
            if top-1>= 0 and grid[left][top-1].is_weight():
                t = top+1
            if bottom < 50 and grid[left][bottom].is_weight():
                b = bottom-1
            if t >= b:
                return weighted
            test = True
#             print("--->", t, b)
            rand = np.random.randint(t, b)
            if left<right and ((right - left<= 3) or (left-1 >= 0 and grid[left-1][bottom].is_neutral() and right < len(grid) and grid[right][bottom].is_neutral())):
                    br = bottom
            if left+1 < right-1:
                br = np.random.randint(left+1, right-1)
            elif left+1 == right-1:
                br = left+1
            else:
                br = right
            vertical = False
        if test == False:
            return weighted
        prob = 3
        if vertical:
            for y in range(top, br):
#                 print(rand, y)
                grid[rand][y].make_weight()
                weighted.append(grid[rand][y])
                draw()
            if not np.random.randint(prob) and br - top <= 2:
                grid[rand][y+1].make_weight()
                weighted.append(grid[rand][y+1])
                draw()
            for y in range(br+1, bottom):
#                 print(rand, y)
                grid[rand][y].make_weight()
                weighted.append(grid[rand][y])
                draw()
        else:
            for x in range(left, br):
                grid[x][rand].make_weight()
                weighted.append(grid[x][rand])
                draw()
            if not np.random.randint(prob) and br - left <= 2:
                grid[x+1][rand].make_weight()
                weighted.append(grid[x+1][rand])
                draw()
            for x in range(br+1, right):
                grid[x][rand].make_weight()
                weighted.append(grid[x][rand])
                draw()
        if vertical:
            weighted += recursive_weight_maze(draw, width, grid, start, end, rand+1, right, top, br, win)
            weighted += recursive_weight_maze(draw, width, grid, start, end, left, rand, top, br, win)
            weighted += recursive_weight_maze(draw, width, grid, start, end, rand+1, right, br+1, bottom, win)
            weighted += recursive_weight_maze(draw, width, grid, start, end, left, rand, br+1, bottom, win)
        else:
            weighted += recursive_weight_maze(draw, width, grid, start, end, left, br, top, rand, win)
            weighted += recursive_weight_maze(draw, width, grid, start, end, left, br, rand+1, bottom, win)
            weighted += recursive_weight_maze(draw, width, grid, start, end, br+1, right, top, rand, win)
            weighted += recursive_weight_maze(draw, width, grid, start, end, br+1, right, rand+1, bottom, win)
    return weighted

In [26]:
def recursive_weight_div(draw, width, grid, start, end, left, right, top, bottom, win, break_at = 0):
    li = []
    for i in range(right):
        grid[0][i].make_weight()
        li.append(grid[0][i])
        grid[bottom-1][right-1-i].make_weight()
        draw()
    for i in range(bottom-1):
        grid[i][0].make_weight()
        li.append(grid[i][0])
        grid[bottom-1-i][right-1].make_weight()
        draw()
    li += recursive_weight_maze(draw, width, grid, start, end, left+1, right-1, top+1, bottom-1, win)
    return li

In [27]:
def recursive_div(draw, width, grid, start, end, left, right, top, bottom, win, break_at = 0):
    for i in range(right):
        grid[0][i].make_barrier()
        grid[bottom-1][right-1-i].make_barrier()
        draw()
    for i in range(bottom-1):
        grid[i][0].make_barrier()
        grid[bottom-1-i][right-1].make_barrier()
        draw()
    recursive_maze(draw, width, grid, start, end, left+1, right-1, top+1, bottom-1, win)

In [28]:
def make_grid(rows, width):
    grid = []
    gap = width // rows
    for i in range(rows):
        grid.append([])
        for j in range(rows):
            node = Node(i, j, gap, rows)
            grid[i].append(node)
    return np.array(grid)

In [29]:
def draw_grid(win, rows, width):
    gap = width // rows
    for i in range(rows+1):
        pygame.draw.line(win, GREY, (0, i*gap), (rows*gap, i*gap))
    for i in range(rows+1):
        pygame.draw.line(win, GREY, (i*gap, 0), (i*gap, rows*gap))

In [30]:
def draw(win, grid, rows, width, algorithms, mazes, options, output, menu = True):
    win.fill(BG_COLOR)
    for row in grid:
        for node in row:
            node.draw(win)
    draw_grid(win, rows, width)
    if menu:
        delta = 700
        ht = 900
        width = ht
        w = 1600
        font = pygame.font.SysFont('comicsans', 35)
        text = font.render("Algorithms", 1, WHITE)
        win.blit(text, (width+delta//6, ht//10 - ht//20 ))
        for algorithm in algorithms:
            algorithm.draw(win, BLACK)
            
        text = font.render("Mazes", 1, WHITE)
        win.blit(text, (width+delta//6, ht//10 + (3*(ht//15)//2) + ht//15 + ht//12 - ht//20 ))
        for maze in mazes:
            maze.draw(win, BLACK)
        
        text = font.render("Options", 1, WHITE)
        win.blit(text, (width+delta//6, ht//10 + (3*(ht//15)//2) + ht//15 + ht//12 + (3*(ht//15)//2) + ht//15 + ht//12 - ht//20 ))
        for option in options:
            option.draw(win, BLACK)
        output.draw(win, (0, 0, 0))
    pygame.display.update()

In [31]:
def get_clicked_pos(pos, rows, width):
    gap = width//rows
    y, x = pos
    row = y // gap
    col = x // gap
    return row, col

In [32]:
def main(win, width):
    ROWS = 50
    w, ht = pygame.display.get_surface().get_size()
    width = ht
    grid = make_grid(ROWS, width)
    delta = w - width
    
    top_start = ht//10
    but_height = ht//15
    but_width = delta//4
    
    algorithms = [
        button(width+delta//5, top_start, but_width-but_height, but_height, "A*"),
        button(width+delta//5+(5*but_width//4), top_start, but_width-but_height, but_height, 'IDA*'),
        button(width+delta//5, top_start+(3*but_height//2), but_width-but_height, but_height, 'Modified A*'),
        button(width+delta//5+(5*but_width//4), top_start+(3*but_height//2), but_width-but_height, but_height, 'DFA*'),
    ]
    top_start = top_start + (3*but_height//2) + but_height + ht//12
    but_height = ht//15
    but_width = delta//4+ delta//10
    mazes = [
        button(width+delta//5, top_start, but_width-delta//8-but_height, but_height, "DFS"),
        button(width+delta//5+(5*(but_width-80)//4), top_start, but_width-but_height, but_height, 'Recursive Division'),
        button(width+delta//5+(5*(but_width-150)//4)//2, top_start+(3*but_height//2), but_width+delta//8-but_height, but_height, 'Modified Sidewinder'),
    ]
    top_start = top_start + (3*but_height//2) + but_height + ht//12
    but_height = ht//15
    but_width =  delta//10
    options = [
        button(width+delta//5, top_start, but_width-but_height+20, but_height, "Clear"),
        button(width+delta//5-20 + delta//5 + (4*(but_width+50)//3), top_start, 0, but_height, "-"),
        button(width+delta//5 + delta//5 + (4*(but_width+50)//3)+(4*(but_width-30)//3), top_start,0, but_height, "+"),
    ]
    sc_start = ht-250
    sc_height = 230
    sc_width = delta-delta//4
    output = screen(width+delta//8, sc_start, sc_width, sc_height, "Choose an Algorithm")
    output.set_label1(f"Number of rows: {ROWS}")
    output.set_text1("1. Pick starting node")
    output.set_text2("2. Pick ending node")
    output.set_text3("3. Choose an algorithm")
    
    start = None
    end = None
    
    run = True
    started = False
    visited = []
    weighted = []
    path = False
    while run:
        
        
        if len(visited):
            visit_animation(visited)
            
        if path:
            path_animation(path)
            
        draw(win, grid, ROWS, width, algorithms, mazes, options, output)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                
            if started:
                continue
                
            
            if pygame.mouse.get_pressed()[0]:
                pos = pygame.mouse.get_pos()
                row,col = get_clicked_pos(pos, ROWS, width)
                if row>= 0 and row<ROWS and col < ROWS and col >= 0:
                    node = grid[row][col]
                    if node in visited:
                        visited.remove(node)
                    if node in weighted:
                        weighted.remove(node)
                    if path:
                        if node in path:
                            path.remove(node)
                    if not start and node != end:
                        if node in visited:
                            visited.remove(node)
                        start = node
                        start.make_start()
                    elif not end and node != start:
                        end = node
                        end.make_end()
                    elif node != end and node != start:
                        node.make_barrier()
                        
                elif algorithms[0].is_hover(pos):
                    output.draw(win, (0, 0, 0))
                    if len(weighted):
                        for node in weighted:
                            node.make_weight()
                    if start and end:
                        for row in grid:
                            for node in row:
                                node.update_neighbors(grid)
                                if not node.is_neutral() and node != start and node != end and not node.is_barrier() and not node.is_weight():
                                    node.reset()
                        visited = []
                        path = []
                        output.set_text1("......")
                        output.set_text2("")
                        output.set_text3("")
                        pygame.display.update()
                        visited, path = A_star(lambda: draw(win, grid, ROWS, width, algorithms, mazes, options, output), grid, start, end, output, win, width)
                
                elif algorithms[1].is_hover(pos):
                    if len(weighted):
                        for node in weighted:
                            node.make_weight()
                    if start and end:
                        for row in grid:
                            for node in row:
                                node.update_neighbors(grid)
                                if not node.is_neutral() and node != start and node != end and not node.is_barrier() and not node.is_weight():
                                    node.reset()
                        visited = []
                        path = []
                        output.set_text1("......")
                        output.set_text2("")
                        output.set_text3("")
                        pygame.display.update()
                        visited, path = IDA_star(lambda: draw(win, grid, ROWS, width, algorithms, mazes, options, output), win, width, output, grid, start, end, h(start.get_pos(), end.get_pos()))
                
                elif algorithms[2].is_hover(pos):
                    if len(weighted):
                        for node in weighted:
                            node.make_weight()
                    if start and end:
                        for row in grid:
                            for node in row:
                                node.update_neighbors(grid)
                                if not node.is_neutral() and node != start and node != end and not node.is_barrier() and not node.is_weight():
                                    node.reset()
                        visited = []
                        path = []
                        output.set_text1("......")
                        output.set_text2("")
                        output.set_text3("")
                        pygame.display.update()
                        visited, path = moded_A_star(lambda: draw(win, grid, ROWS, width, algorithms, mazes, options, output), grid, start, end, output, win, width)
                
                elif algorithms[3].is_hover(pos):
                    if len(weighted):
                        for node in weighted:
                            node.make_weight()
                    if start and end:
                        for row in grid:
                            for node in row:
                                node.update_neighbors(grid)
                                if not node.is_neutral() and node != start and node != end and not node.is_barrier() and not node.is_weight():
                                    node.reset()
                        visited = []
                        path = []
                        output.set_text1("......")
                        output.set_text2("")
                        output.set_text3("")
                        pygame.display.update()
                        visited, path = DFA_star(lambda: draw(win, grid, ROWS, width, algorithms, mazes, options, output), grid, start, end, output, win, width)
                
                elif mazes[0].is_hover(pos):
                    output.set_text1("......")
                    output.set_text2("")
                    output.set_text3("")
                    pygame.display.update()
                    start = None
                    end = None
                    visited = []
                    path = []
                    weighted = []
                    for row in grid:
                        for node in row:
                            if node.is_barrier() or node.is_start() or node.is_end():
                                node.reset()
                    dfs_maze(lambda: draw(win, grid, ROWS, width, algorithms, mazes, options, output), width, grid, start, end, 0, ROWS, 0, ROWS, win)
                    output.set_text1("1. Pick starting node")
                    output.set_text2("2. Pick ending node")
                    output.set_text3("3. Choose an algorithm")
                    
                elif mazes[1].is_hover(pos):
                    output.set_text1("......")
                    output.set_text2("")
                    output.set_text3("")
                    pygame.display.update()
                    start = None
                    end = None
                    visited = []
                    path = []
                    weighted = []
                    for row in grid:
                        for node in row:
#                             if node.is_barrier() or node.is_start() or node.is_end():
                            node.reset()
                    recursive_div(lambda: draw(win, grid, ROWS, width, algorithms, mazes, options, output),width, grid, start, end, 0, ROWS, 0, ROWS, win)
                    output.set_text1("1. Pick starting node")
                    output.set_text2("2. Pick ending node")
                    output.set_text3("3. Choose an algorithm")
            
                elif mazes[2].is_hover(pos):
                    output.set_text1("......")
                    output.set_text2("")
                    output.set_text3("")
                    pygame.display.update()
                    start = None
                    end = None
                    visited = []
                    path = []
                    weighted = []
                    for row in grid:
                        for node in row:
                            if node.is_barrier() or node.is_start() or node.is_end():
                                node.reset()
                    moded_sidewinder_maze(lambda: draw(win, grid, ROWS, width, algorithms, mazes, options, output), width, grid, start, end, 0, ROWS, 0, ROWS, win)
                    output.set_text1("1. Pick starting node")
                    output.set_text2("2. Pick ending node")
                    output.set_text3("3. Choose an algorithm")
                    
                elif options[0].is_hover(pos):
                    output.set_text1("1. Pick starting node")
                    output.set_text2("2. Pick ending node")
                    output.set_text3("3. Choose an algorithm")
                    pygame.display.update()
                    weighted = []
                    start = None
                    end = None
                    visited = []
                    path = []
                    weighted = []
                    for row in grid:
                        for node in row:
                            node.reset()
                            
                elif options[1].is_hover(pos):
                    weighted = []
                    start = None
                    end = None
                    visited = []
                    path = []
                    weighted = []
                    for row in grid:
                        for node in row:
                            node.reset()
                    if ROWS>5:
                        ROWS-=1
                        grid = make_grid(ROWS, width)
                    output.set_label1(f"Number of rows: {ROWS}")
                        
                elif options[2].is_hover(pos):
                    weighted = []
                    start = None
                    end = None
                    visited = []
                    path = []
                    weighted = []
                    for row in grid:
                        for node in row:
                            node.reset()
                    if ROWS<50:
                        ROWS+=1
                        grid = make_grid(ROWS, width)
                    output.set_label1(f"Number of rows: {ROWS}")
                        
            elif pygame.mouse.get_pressed()[2]:
                pos = pygame.mouse.get_pos()
                row,col = get_clicked_pos(pos, ROWS, width)
                if row>= 0 and row<ROWS and col < ROWS and col >= 0:
                    node = grid[row][col]
                    if path:
                        if node in path:
                            path.remove(node)
                    if node in visited:
                        visited.remove(node)
                    if node in weighted:
                        weighted.remove(node)
                    node.reset()
                    if node == start:
                        start = None
                    elif node == end:
                        end = None
            
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    run = False
            
#             if event.type == pygame.KEYDOWN:
                
# #                 if event.key == pygame.K_w:
# #                     pos = pygame.mouse.get_pos()
# #                     row,col = get_clicked_pos(pos, ROWS, width)
# #                     if row>= 0 and row<ROWS and col < ROWS and col >= 0:
# #                         node = grid[row][col]
# #                         if node in visited:
# #                             visited.remove(node)
# #                         if path:
# #                             if node in path:
# #                                 path.remove(node)
# #                         if not start and node != end:
# #                             start = node
# #                             start.make_start()
# #                         elif not end and node != start:
# #                             end = node
# #                             end.make_end()
# #                         elif node != end and node != start and not node.is_barrier():
# #                             weighted.append(node)
# #                             node.make_weight()
                
#                 if event.key == pygame.K_i and not started:
#                     if len(weighted):
#                         for node in weighted:
#                             node.make_weight()
#                     if start and end:
#                         for row in grid:
#                             for node in row:
#                                 node.update_neighbors(grid)
#                                 if not node.is_neutral() and node != start and node != end and not node.is_barrier() and not node.is_weight():
#                                     node.reset()
#                         visited = []
#                         path = []
#                         visited, path = IDA_star(lambda: draw(win, grid, ROWS, width, algorithms, mazes, options, output), win, width, output, grid, start, end, h(start.get_pos(), end.get_pos()))
                
#                 if event.key == pygame.K_a and not started:
#                     if len(weighted):
#                         for node in weighted:
#                             node.make_weight()
#                     if start and end:
#                         for row in grid:
#                             for node in row:
#                                 node.update_neighbors(grid)
#                                 if not node.is_neutral() and node != start and node != end and not node.is_barrier() and not node.is_weight():
#                                     node.reset()
#                         visited = []
#                         path = []
#                         visited, path = A_star(lambda: draw(win, grid, ROWS, width, algorithms, mazes, options, output), grid, start, end, output)
                
#                 if event.key == pygame.K_m and not started:
#                     if len(weighted):
#                         for node in weighted:
#                             node.make_weight()
#                     if start and end:
#                         for row in grid:
#                             for node in row:
#                                 node.update_neighbors(grid)
#                                 if not node.is_neutral() and node != start and node != end and not node.is_barrier() and not node.is_weight():
#                                     node.reset()
#                         visited = []
#                         path = []
#                         visited, path = moded_A_star(lambda: draw(win, grid, ROWS, width, algorithms, mazes, options, output), grid, start, end, output)
                        
#                 if event.key == pygame.K_d and not started:
#                     if len(weighted):
#                         for node in weighted:
#                             node.make_weight()
#                     if start and end:
#                         for row in grid:
#                             for node in row:
#                                 node.update_neighbors(grid)
#                                 if not node.is_neutral() and node != start and node != end and not node.is_barrier() and not node.is_weight():
#                                     node.reset()
#                         visited = []
#                         path = []
#                         visited, path = DFA_star(lambda: draw(win, grid, ROWS, width, algorithms, mazes, options, output), grid, start, end, output)
            
#                 if event.key == pygame.K_LEFT and not started:
#                     start = None
#                     end = None
#                     visited = []
#                     path = []
#                     weighted = []
#                     for row in grid:
#                         for node in row:
#                             if node.is_barrier() or node.is_start() or node.is_end():
#                                 node.reset()
#                     dfs_maze(lambda: draw(win, grid, ROWS, width, algorithms, mazes, options, output), width, grid, start, end, 0, ROWS, 0, ROWS, win)
                
#                 if event.key == pygame.K_UP and not started:
#                     start = None
#                     end = None
#                     visited = []
#                     path = []
#                     weighted = []
#                     for row in grid:
#                         for node in row:
#                             if node.is_barrier() or node.is_start() or node.is_end():
#                                 node.reset()
#                     moded_sidewinder_maze(lambda: draw(win, grid, ROWS, width, algorithms, mazes, options, output), width, grid, start, end, 0, ROWS, 0, ROWS, win)
                
#                 if event.key == pygame.K_DOWN and not started:
#                     start = None
#                     end = None
#                     visited = []
#                     path = []
#                     weighted = []
#                     for row in grid:
#                         for node in row:
#                             node.reset()
#                     recursive_div(lambda: draw(win, grid, ROWS, width, algorithms, mazes, options, output), width, grid, start, end, 0, ROWS, 0, ROWS, win)
# #                     weighted = recursive_weight_div(lambda: draw(win, grid, ROWS, width, algorithms, mazes, options, output), width, grid, start, end, 0, ROWS, 0, ROWS)
                
#                 if event.key == pygame.K_c and not started:
#                     weighted = []
#                     start = None
#                     end = None
#                     visited = []
#                     path = []
#                     weighted = []
#                     for row in grid:
#                         for node in row:
#                             node.reset()
            
    pygame.quit()

In [33]:
pygame.init()
WIDTH = 950
WIN = pygame.display.set_mode((WIDTH+700, WIDTH), pygame.FULLSCREEN)

pygame.display.set_caption("Path Finder")
main(WIN, WIDTH)